In [1]:
import warnings
import os
import pandas as pd
import numpy as np
import spacy
warnings.filterwarnings("ignore")

# Import your DI container and components
from DependencyInjection.container import Container

In [2]:
# Initialize the container
container = Container()

def extract_resume_text(file_path):
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError('File not found')

        if file_path.split()[-1].endswith('.pdf'):
            extractor = container.pdf_extractor()
        elif file_path.split()[-1].endswith(('.doc', '.docx')):
            extractor = container.word_extractor()
        else:
            return "Error: Unsupported file format (only PDF/DOC/DOCX accepted)"

        extracted_text = extractor.extract_text(file_path)

        if not extracted_text:
            return "Error: No Text could be Extracted from file"

        return extracted_text

    except Exception as e:
        return f'Error: {e}'

In [3]:
"""
Why spaCy?
More accurate tokenization (handles hyphenated words, contractions, and punctuation better).
Built-in lemmatization (no need for additional libraries like NLTK's WordNet).
Faster (optimized Cython backend).
Scalable (supports custom pipelines for resume-specific terms).
"""
#  spaCy (better for resume parsing since it includes NER and POS out-of-the-box)
nlp = spacy.load('en_core_web_sm')

def process_text_with_spacy(text):
    doc = nlp(text)

    # Preprocessing of Text:
    #Tokenize the Text, Excluding Stop-words and Punctuations:
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and not token.is_space]


    # Lemmatization:
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

    # Named Entity Recognition
    entities = {ent.text: ent.label_ for ent in doc.ents}


    return {"tokens": tokens, "lemmas": lemmas, "entities": entities}

In [4]:
# Clean and Read the Resume and Description Data from the CSV's
resumeDF = pd.read_csv('data/Resume.csv', nrows = 100)
jdDF = pd.read_csv('data/fake_job_postings.csv', nrows = 100)

In [5]:
# Inspect the Values of both the DataFrames.
resumeDF.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [6]:
jdDF.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [7]:
resumeDF.info()
# Obbservations:
# No Null Values Present

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           100 non-null    int64 
 1   Resume_str   100 non-null    object
 2   Resume_html  100 non-null    object
 3   Category     100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [8]:
jdDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               100 non-null    int64 
 1   title                100 non-null    object
 2   location             100 non-null    object
 3   department           27 non-null     object
 4   salary_range         13 non-null     object
 5   company_profile      88 non-null     object
 6   description          100 non-null    object
 7   requirements         89 non-null     object
 8   benefits             54 non-null     object
 9   telecommuting        100 non-null    int64 
 10  has_company_logo     100 non-null    int64 
 11  has_questions        100 non-null    int64 
 12  employment_type      83 non-null     object
 13  required_experience  61 non-null     object
 14  required_education   67 non-null     object
 15  industry             78 non-null     object
 16  function 

In [9]:
jdDF['requirements'] = jdDF['requirements'].fillna('').astype(str)

In [10]:
resume_text = []
jd_text = []

for i, row in resumeDF.iterrows():
    text = row['Resume_str'] + '\n\n'
    resume_text.append(text)

for i , row in jdDF.iterrows():
    text = row['title'] + '\n\n' + row['description'] + '\n\n' + row['requirements']
    jd_text.append(text)

In [11]:
'''
if __name__ == '__main__':
    file_path = "data/AllResumes.pdf"

    # Process the file
    resume = extract_resume_text(file_path)

    # Print results
    if resume.startswith("Error"):
        print(resume)
    else:
        print("Successfully extracted text:\n")
        print(resume)
'''

'\nif __name__ == \'__main__\':\n    file_path = "data/AllResumes.pdf"\n\n    # Process the file\n    resume = extract_resume_text(file_path)\n\n    # Print results\n    if resume.startswith("Error"):\n        print(resume)\n    else:\n        print("Successfully extracted text:\n")\n        print(resume)\n'

In [12]:
'''
job_description_path = 'data/Machine_Learning_Engineer_Job_Description.pdf'
jd = extract_resume_text(job_description_path).split('Job Description:', 1)[-1].strip()
print(jd)
'''

"\njob_description_path = 'data/Machine_Learning_Engineer_Job_Description.pdf'\njd = extract_resume_text(job_description_path).split('Job Description:', 1)[-1].strip()\nprint(jd)\n"

In [13]:
# To Tackle the Memory Error while processing the large texts we divide the text into chunks.
'''
def process_text_in_chunks(texts, batch_size):
    results = []
    for i in range(0, len(texts), batch_size):
        batch_text = " ".join(texts[i : i + batch_size])
        results.append(process_text_with_spacy(batch_text))
    return results
'''

'\ndef process_text_in_chunks(texts, batch_size):\n    results = []\n    for i in range(0, len(texts), batch_size):\n        batch_text = " ".join(texts[i : i + batch_size])\n        results.append(process_text_with_spacy(batch_text))\n    return results\n'

In [14]:
resume_processed = [process_text_with_spacy(text) for text in resume_text]

In [15]:
description_processed = [process_text_with_spacy(text) for text in jd_text]

In [16]:
'''
print("Resume Tokens:",  resume_processed["tokens"])
print("Resume Lemmas:", resume_processed["lemmas"])
print("Resume Entities:", resume_processed["entities"])
'''

'\nprint("Resume Tokens:",  resume_processed["tokens"])\nprint("Resume Lemmas:", resume_processed["lemmas"])\nprint("Resume Entities:", resume_processed["entities"])\n'

In [17]:
'''
print("JD Tokens:",  description_processed["tokens"])
print("JD Lemmas:", description_processed["lemmas"])
print("JD Entities:", description_processed["entities"])
'''

'\nprint("JD Tokens:",  description_processed["tokens"])\nprint("JD Lemmas:", description_processed["lemmas"])\nprint("JD Entities:", description_processed["entities"])\n'

In [18]:
model = container.ml_models()

In [19]:
resume_tokens_list = [doc["tokens"] for doc in resume_processed]
jd_tokens_list = [doc["tokens"] for doc in description_processed]
all_resume_tokens = [token for tokens in resume_tokens_list for token in tokens]
all_jd_tokens = [token for tokens in jd_tokens_list for token in tokens]

word_weights = model.load_TfIdfVectorizer(all_resume_tokens, all_jd_tokens)
print(word_weights)

{'$ 200': np.float64(1.4054651081081644), '$ 200 cost': np.float64(1.4054651081081644), '+ usd': np.float64(1.4054651081081644), '+ usd monthly': np.float64(1.4054651081081644), '01/1999': np.float64(1.4054651081081644), '01/1999 company': np.float64(1.4054651081081644), '01/1999 company city': np.float64(1.4054651081081644), '01/2000': np.float64(1.4054651081081644), '01/2001': np.float64(1.4054651081081644), '01/2002': np.float64(1.4054651081081644), '01/2004': np.float64(1.4054651081081644), '01/2007': np.float64(1.4054651081081644), '01/2008': np.float64(1.4054651081081644), '01/2009': np.float64(1.4054651081081644), '01/2010': np.float64(1.4054651081081644), '01/2012': np.float64(1.4054651081081644), '01/2013': np.float64(1.4054651081081644), '01/2014': np.float64(1.4054651081081644), '02/2012': np.float64(1.4054651081081644), '02/2012 company': np.float64(1.4054651081081644), '02/2012 company city': np.float64(1.4054651081081644), '02/2013': np.float64(1.4054651081081644), '02/20

In [ ]:
resume_embeddings = []
jd_embeddings = []

#Load the S-BERT Model to get the Weighted Embeddings.
sbert_model = model.load_sbert_model()

for tokens in resume_tokens_list:
    embed = model.get_weighted_embeddings_from_sbert(tokens, word_weights, sbert_model)
    resume_embeddings.append(embed)

for tokens in jd_tokens_list:
    embed = model.get_weighted_embeddings_from_sbert(tokens, word_weights, sbert_model)
    jd_embeddings.append(embed)

resume_embeddings = np.array(resume_embeddings)
jd_embeddings = np.array(jd_embeddings)

In [21]:
# Train the Resume Scoring Model
rf_model = model.train_scoring_model(resume_embeddings, jd_embeddings)

X shape: (50, 3)
Scores shape: (50,)
Train MSE: 0.00423231060405445
Test MSE: 0.006465293453156952
Train R2: 0.9889219979298057
Test R2: 0.9847092789323548


In [22]:
# Testing the Prediction Capacity of the Model:
# Extract text
resume_text_new = extract_resume_text("data/Rahul Victor Sunkara_UpdatedResume.pdf")
jd_text_new = extract_resume_text("data/FashionDesignerJD.pdf")

# Preprocess
resume_tokens_new = process_text_with_spacy(resume_text_new)["tokens"]
jd_tokens_new = process_text_with_spacy(jd_text_new)["tokens"]

# Generate embeddings
word_weights = model.load_TfIdfVectorizer(resume_tokens_new, jd_tokens_new)
resume_embed_new = model.get_weighted_embeddings_from_sbert(resume_tokens_new, word_weights)
jd_embed_new = model.get_weighted_embeddings_from_sbert(jd_tokens_new, word_weights)

# Predict score
score = model.predict_score(resume_embed_new, jd_embed_new, rf_model)
print(f"Predicted Match Score: {score:.1f}/100")

ValueError: Expected 2D array, got 1D array instead:
array=[-2.75613924e-02  2.16123672e-02 -1.77339060e-02  1.61657573e-02
 -2.59013305e-02 -1.52357459e-02  6.27015269e-02  1.60088971e-02
 -4.42703922e-03  7.29167533e-04  1.33913817e-02 -1.83818255e-02
  1.28576959e-02  4.90252333e-03 -2.15056141e-02  5.45038637e-03
  1.61617524e-03 -1.58126602e-02 -5.30738654e-02 -4.02332654e-02
 -4.64961330e-02  8.12994636e-05 -1.87387798e-02  9.42223861e-03
 -7.47445236e-03  3.52637773e-02 -1.49947834e-02  1.71869382e-02
  2.94694134e-02 -8.25217143e-02 -6.56720532e-03  1.94922627e-02
  3.35523176e-02  1.78561075e-03 -2.55452318e-02  1.26646188e-02
  6.74324958e-03 -7.08477020e-03  4.07882986e-03 -1.02708876e-02
 -2.75127888e-02 -5.68112652e-02 -7.08124799e-04  2.07724144e-03
  1.82567078e-02  2.20722997e-04  2.97187595e-03  1.22966265e-04
  1.48693964e-03  1.84336208e-02 -3.87698520e-02 -2.74498413e-02
 -2.95290473e-02  4.37495728e-03  1.15116140e-02  1.42185857e-02
 -8.23449428e-03  4.13096098e-0